In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import json

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
#from sklearn.model_selection import train_test_split

from packaging import version
import pytorch_lightning as pl
from pytorch_lightning import Callback

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.optim as optimizers

from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
if version.parse(pl.__version__) < version.parse("1.0.2"):
    raise RuntimeError("PyTorch Lightning>=1.0.2 is required for this example.")

In [3]:
all_new = pd.read_csv('data/feature_df.csv')
wordvec_df = pd.read_csv('sub_notebook/data/wordvectorized_df.csv')
bert_embeded = pd.read_csv('data/bert_embeded.csv')
bert_svd256 = pd.read_csv("data/bert_svd256.csv")
ALBERT_svd256 = pd.read_csv("data/ALBERT_svd256.csv")
ROBERTA_svd256 = pd.read_csv("data/ROBERTA_svd256.csv")
XLMROBERTA_svd256 = pd.read_csv("data/XLMROBERTA_svd256.csv")
tfidf_svd_raw_64 = pd.read_csv("data/tfidf_svd_raw_64.csv")
mfw_df = pd.read_csv('data/mfw.csv')
miw_df = pd.read_csv('data/miw.csv')

In [4]:
#all_new = pd.merge(all_new, bert_embeded, on="id", how="outer")
all_new = pd.merge(all_new, XLMROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, tfidf_svd_raw_64, on="id", how="outer")
all_new = pd.merge(all_new, wordvec_df, on="id", how="outer")
all_new = pd.merge(all_new, miw_df, on="id", how="outer")
all_new = pd.merge(all_new, mfw_df.drop(["mfw", "sfw"], axis=1), on="id", how="outer")

In [5]:
test_data = all_new[all_new["data_type"] == "test"]
train_data = all_new[all_new["data_type"] == "train"]

In [6]:
with open("data/lgbm_config_xlmroberta256.json",) as f:
    config_dict = json.load(f)

target_cols = ["state"]
feature_cols = config_dict["selected_feature"]
CATEGORICAL_COLS = [
    "country",
    "category1",
    "category2",
    "country+category1",
    "country+category2",
    "category1+category2",
]

for categorical_col in CATEGORICAL_COLS:
    if categorical_col in feature_cols:
        feature_cols.remove(categorical_col)

In [7]:
def check_nan_columns(df, feature_cols):
    col_nan = set(np.where(np.isnan(np.array(df[feature_cols])) ==True)[1])
    col_names = df[feature_cols].columns
    col_nan = col_names[list(col_nan)]
    print(f"there are {len(col_nan)} columns including np.nan")
    print(f"{list(col_nan)}")
    
    for col in col_nan:
        feature_cols.remove(col)
        
    return feature_cols

In [8]:
feature_cols = check_nan_columns(all_new, feature_cols)

there are 6 columns including np.nan
['agg_std_duration_grpby_category2', 'agg_std_duration+number_of_<video>_grpby_category2', 'agg_std_goal_min/number_of_<div>_grpby_category2', 'agg_std_goal_min/number_of_<a>_grpby_category2', 'agg_std_duration/goal_min_grpby_country+category2', 'agg_std_duration+number_of_<span>_grpby_country+category2']


In [9]:
class CFDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_file = csv_file
        self.transform = transform
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        
        
    def __len__(self):
        return len(self.csv_file)
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        features = self.csv_file[self.feature_cols].iloc[idx]
        features = torch.FloatTensor(features)
        
        target = self.csv_file[self.target_cols].iloc[idx]
        target = torch.tensor(target)
        
        if self.transform:
            features = self.transform(features)
            
        return features, target

In [10]:
class CFDataModule(pl.LightningDataModule):
    def __init__(self, train, validation, transform, batch_size, num_workers):
        super().__init__()
        self.csv_file = train
        self.transform = transform
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.validation = validation
        

    def setup(self, stage=None):
        self.train_dataset = CFDataset(csv_file=self.csv_file, transform=self.transform)
        self.val_dataset = CFDataset(csv_file=self.validation, transform=self.transform)
            
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          #pin_memory=True,
                         )
    
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          #pin_memory=True,
                         )

In [11]:
class CFModule(pl.LightningModule):
    def __init__(self, num_features, num_classes, trial):
        super(CFModule, self).__init__()
        self.layers = []
        self.dense_layers = []
        self.dropouts = []

        # We optimize the number of layers, hidden units in each layer and dropouts.
        n_layers = trial.suggest_int("n_layers", 2, 4)
        dropout = trial.suggest_float("dropout", 0.2, 0.5)
        input_dim = num_features
        for i in range(n_layers):
            output_dim = trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True)
            self.layers.append(nn.BatchNorm1d(input_dim))
            self.dropouts.append(nn.Dropout(dropout))
            self.dense_layers.append(nn.utils.weight_norm(nn.Linear(input_dim, output_dim)))
            input_dim = output_dim

        self.layers.append(nn.Linear(input_dim, num_classes))

        # Assigning the layers as class variables (PyTorch requirement).
        # Parameters of a layer are returned when calling model.parameters(),
        # only if the layer is a class variable. Thus, assigning as class
        # variable is necessary to make the layer parameters trainable.
        for idx, layer in enumerate(self.layers):
            setattr(self, "fc{}".format(idx), layer)

        # Assigning the dropouts as class variables (PyTorch requirement), for
        # the same reason as above.
        for idx, dropout in enumerate(self.dropouts):
            setattr(self, "drop{}".format(idx), dropout)
            
        for idx, dense in enumerate(self.dense_layers):
            setattr(self, "dense{}".format(idx), dense)
            
    
    def forward(self, x):
        for layer, dropout, dense_layer in zip(self.layers, self.dropouts, self.dense_layers):
            x = layer(x)
            x = dropout(x)
            x = F.relu(dense_layer(x))
        x = self.layers[-1](x)
        
        return torch.sigmoid(x)
    
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        # you should define log as {"tag_name/log_name"}
        tensorboard_logs = {'train/train_loss': loss, "train/train_acc": acc}
        return {"loss": loss, "acc": acc, "logs": tensorboard_logs, "progress_bar": tensorboard_logs}
    
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        logs = {"val_loss": loss, "val_acc": acc}
        self.log("vall_loss", loss)
        return {"val_loss": loss, "val_acc": acc, "progress_bar": logs}

    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val/avg_loss': avg_loss, "val/avg_acc": avg_acc}
        #print(f"val_loss: {avg_loss}, val_acc: {avg_acc}")
        # you should call back as name "val_loss" to using the Early-Stopping
        return {'val_loss': avg_loss, "val_acc": avg_acc, 'log': tensorboard_logs}
    
    
    def test_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        return {"preds": pred}
    
    
    def test_epoch_end(self, outputs):
        preds = torch.cat([x["preds"] for x in outputs], dim=0).squeeze()
        preds = np.array(preds.tolist())
        return {"preds": preds}
    
    
    def configure_optimizers(self):
        optimizer = optimizers.Adam(self.parameters(), 
                                    lr=0.01, betas=(0.9,0.999),
                                    eps=1e-08, weight_decay=0, amsgrad=False)
        scheduler = {"scheduler": 
                     optimizers.lr_scheduler.CosineAnnealingLR(
                        optimizer, T_max=10),
                    "interval": "epoch",
                    "monitor": "val_loss"}
        return [optimizer], [scheduler]
    
    
    def criterion(self, pred, t):
        #pred = pred.view(-1)
        pred = pred.float()
        t = t.float()
        return F.binary_cross_entropy(input=pred, target=t)
    
    
    def metric(self, pred, t):
        t = t.to('cpu')
        pred = np.array(pred.tolist())
        pred = np.where(pred<0.5, 0, 1)
        return f1_score(y_true=t, y_pred=pred, average='binary', sample_weight=None, zero_division='warn')

In [12]:
class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [13]:
def objective(trial):
    
    # data module config
    seed = 123456
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    for fold, (t_idx, v_idx) in enumerate(cv.split(train_data)):
        train_set = train_data.loc[t_idx]
        val_set = train_data.loc[v_idx]
    
    transform = None
    batch_size = 256*4
    num_workers = 4
    PERCENT_VALID_EXAMPLES = 1
    EPOCHS = 15
    
    # model config
    num_input = len(feature_cols)
    num_classes = 1
    
    cf = CFDataModule(train=train_set, validation=val_set,
                      transform=transform, batch_size=batch_size, num_workers=num_workers)
    model = CFModule(num_features=num_input, num_classes=num_classes, trial=trial)
    
    
    # Filenames for each trial must be made unique in order to access each checkpoint.
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss"
    )

    # The default logger in PyTorch Lightning writes to event files to be consumed by
    # TensorBoard. We don't use any logger here as it requires us to implement several abstract
    # methods. Instead we setup a simple callback, that saves metrics from each validation step.
    metrics_callback = MetricsCallback()
    trainer = pl.Trainer(
        logger=False,
        limit_val_batches=PERCENT_VALID_EXAMPLES,
        checkpoint_callback=checkpoint_callback,
        max_epochs=EPOCHS,
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, PyTorchLightningPruningCallback(trial, monitor="val_loss")],
    )
    
    trainer.fit(model, cf)

    return metrics_callback.metrics[-1]["val_loss"].item()

In [14]:
if __name__ == "__main__":
    
    DIR = os.getcwd()
    MODEL_DIR = os.path.join(DIR, "result")
    
    pruner = optuna.pruners.NopPruner()

    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(objective, n_trials=50, timeout=int(1*60*60))

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    shutil.rmtree(MODEL_DIR)

[I 2021-01-28 17:52:22,944] A new study created in memory with name: no-name-2f499f82-1889-4cb9-b140-31a0e6542af8
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 622   
1 | fc1    | BatchNorm1d | 10    
2 | fc2    | BatchNorm1d | 16    
3 | fc3    | Linear      | 101   
4 | drop0  | Dropout     | 0     
5 | drop1  | Dropout     | 0     
6 | drop2  | Dropout     | 0     
7 | dense0 | Linear      | 1.6 K 
8 | dense1 | Linear      | 56    
9 | dense2 | Linear      | 1.0 K 
---------------------------------------
3.4 K     Trainable params
0         Non-trainable params
3.4 K     Total params


/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


[I 2021-01-28 17:55:59,051] Trial 0 finished with value: 0.5125813484191895 and parameters: {'n_layers': 3, 'dropout': 0.46903847223365513, 'n_units_l0': 5, 'n_units_l1': 8, 'n_units_l2': 100}. Best is trial 0 with value: 0.5125813484191895.


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name   | Type        | Params
----------------------------------------
0  | fc0    | BatchNorm1d | 622   
1  | fc1    | BatchNorm1d | 16    
2  | fc2    | BatchNorm1d | 84    
3  | fc3    | BatchNorm1d | 12    
4  | fc4    | Linear      | 24    
5  | drop0  | Dropout     | 0     
6  | drop1  | Dropout     | 0     
7  | drop2  | Dropout     | 0     
8  | drop3  | Dropout     | 0     
9  | dense0 | Linear      | 2.5 K 
10 | dense1 | Linear      | 420   
11 | dense2 | Linear      | 264   
12 | dense3 | Linear      | 184   
----------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params


...

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 622   
1 | fc1    | BatchNorm1d | 28    
2 | fc2    | Linear      | 14    
3 | drop0  | Dropout     | 0     
4 | drop1  | Dropout     | 0     
5 | dense0 | Linear      | 4.4 K 
6 | dense1 | Linear      | 208   
---------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params


[I 2021-01-28 18:49:19,526] Trial 11 finished with value: 0.4889187812805176 and parameters: {'n_layers': 2, 'dropout': 0.2082440584767325, 'n_units_l0': 14, 'n_units_l1': 13}. Best is trial 6 with value: 0.48797258734703064.


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 622   
1 | fc1    | BatchNorm1d | 86    
2 | fc2    | Linear      | 13    
3 | drop0  | Dropout     | 0     
4 | drop1  | Dropout     | 0     
5 | dense0 | Linear      | 13.5 K
6 | dense1 | Linear      | 540   
---------------------------------------
14.7 K    Trainable params
0         Non-trainable params
14.7 K    Total params


[I 2021-01-28 18:54:43,192] Trial 12 finished with value: 0.48151499032974243 and parameters: {'n_layers': 2, 'dropout': 0.27116122707293966, 'n_units_l0': 43, 'n_units_l1': 12}. Best is trial 12 with value: 0.48151499032974243.



Number of finished trials: 13
Best trial:
  Value: 0.48151499032974243
  Params: 
    n_layers: 2
    dropout: 0.27116122707293966
    n_units_l0: 43
    n_units_l1: 12


In [15]:
config_dict = {
    "selected_feature": feature_cols,
    "best_params": trial.params,
}

In [16]:
with open("data/nn_optuna_config_xlmroberta256.json", 'w') as f:
    json.dump(config_dict, f)